In [ ]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np


from pandas_profiling import ProfileReport

import time
!pip install apyori
from apyori import apriori as apriori1
!pip install mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori as apriori2
from mlxtend.frequent_patterns import association_rules
# from mlxtend.frequent_patterns import fpgrowth

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
url1 = 'https://raw.githubusercontent.com/ikanurlaily/market-basket2/main/association_order.csv'
association_order = pd.read_csv(url1, error_bad_lines=False)
association_order.head(3)

,orderid,itemid
0,31379820545759,719740607
1,31378575577269,1825360194
2,31369591568249,1108903291


In [ ]:
url2 = 'https://raw.githubusercontent.com/ikanurlaily/market-basket2/main/rules.csv'
rules = pd.read_csv(url2, error_bad_lines=False)
rules.head(3)

,rule
0,100242812>80361758
1,100242812>89031406
2,1003153762>1016449477


In [ ]:
grouped_df = association_order.groupby("orderid")
order_gr = grouped_df["itemid"].apply(list)
order_gr = order_gr.reset_index()
order_gr = DataFrame(order_gr)
order_gr.head(3)
#order_gr.to_csv('ordergroup.csv')

,orderid,itemid
0,31338000509845,"[2662778119, 2255381856, 1825056968, 575945551..."
1,31338001158725,"[6112866826, 5013004529, 5712867361]"
2,31338002167561,"[5712106216, 3310344299, 6303964568, 331073883..."


In [ ]:
order_gr['itemid'] = order_gr['itemid'].astype(str).str.replace(r"[", '')
order_gr['itemid'] = order_gr['itemid'].astype(str).str.replace(r"]", '')
order_gr['itemid'] = order_gr['itemid'].astype(str).str.replace(r" ", '')

In [ ]:
order_gr.head(3)

,orderid,itemid
0,31338000509845,"2662778119,2255381856,1825056968,575945551,144..."
1,31338001158725,"6112866826,5013004529,5712867361"
2,31338002167561,"5712106216,3310344299,6303964568,3310738830,38..."


In [ ]:
def split_list_like(df: pd.DataFrame, col: str, new_col_prefix: str, pat: str = None):
    df = df.copy()
    split_col = df[col].str.split(pat, expand=True)

    return df.assign(
        **{
            f"{new_col_prefix}_{x}": split_col.iloc[:, x]
            for x in range(split_col.shape[1])
        }
    )
split = order_gr.pipe(
    split_list_like,
    col="itemid",
    new_col_prefix="itemid",
    pat=",",
)

In [ ]:
split.head(3)

,orderid,itemid,itemid_0,itemid_1,itemid_2,itemid_3,itemid_4,itemid_5,itemid_6,itemid_7,itemid_8,itemid_9,itemid_10,itemid_11,itemid_12,itemid_13,itemid_14,itemid_15,itemid_16,itemid_17,itemid_18,itemid_19,itemid_20,itemid_21,itemid_22,itemid_23,itemid_24,itemid_25,itemid_26,itemid_27,itemid_28,itemid_29,itemid_30,itemid_31,itemid_32,itemid_33,itemid_34,itemid_35,itemid_36,itemid_37,itemid_38,itemid_39
0,31338000509845,"2662778119,2255381856,1825056968,575945551,144...",2662778119,2255381856,1825056968,575945551,1444935832,1688963479,391098483,6807602156,382577821,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,31338001158725,"6112866826,5013004529,5712867361",6112866826,5013004529,5712867361,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,31338002167561,"5712106216,3310344299,6303964568,3310738830,38...",5712106216,3310344299,6303964568,3310738830,3802384595,4101882142,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
split = split.drop(columns=['itemid'])
split = split.drop(columns=['orderid'])
split.head(3)

,itemid_0,itemid_1,itemid_2,itemid_3,itemid_4,itemid_5,itemid_6,itemid_7,itemid_8,itemid_9,itemid_10,itemid_11,itemid_12,itemid_13,itemid_14,itemid_15,itemid_16,itemid_17,itemid_18,itemid_19,itemid_20,itemid_21,itemid_22,itemid_23,itemid_24,itemid_25,itemid_26,itemid_27,itemid_28,itemid_29,itemid_30,itemid_31,itemid_32,itemid_33,itemid_34,itemid_35,itemid_36,itemid_37,itemid_38,itemid_39
0,2662778119,2255381856,1825056968,575945551,1444935832,1688963479,391098483,6807602156,382577821,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,6112866826,5013004529,5712867361,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,5712106216,3310344299,6303964568,3310738830,3802384595,4101882142,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
split.fillna(0,inplace=True)

In [ ]:
df = split

In [ ]:
transactions = []
for i in range(0,len(df)):
    transactions.append([str(df.values[i,j]) for j in range(0,39) if str(df.values[i,j])!='0'])

In [ ]:
transactions[0]

['2662778119',
 '2255381856',
 '1825056968',
 '575945551',
 '1444935832',
 '1688963479',
 '391098483',
 '6807602156',
 '382577821']

In [ ]:
rules = apriori(transactions)


In [ ]:
Results = list(rules)
Results

[]

In [ ]:
df_results = pd.DataFrame(Results)

In [ ]:
df_results.head()

""


In [ ]:
#ref : https://www.kaggle.com/whs2018/marker-analysis